In [1]:
import time
import requests
headers = {'User-Agent': 'WeRead/1.5.8 WRBrand/Meizu Dalvik/2.1.0 (Linux; U; Android 5.1; M3s Build/LMY47I)',\
           'accessToken': 'QaJPvQ4t',\
           'vid': '7649467'}
from pymongo import MongoClient
client = MongoClient()
db = client.weread
books = db.books
users = db.users
categories = db.categories
booksrd = db.booksrd
from py2neo.ogm import GraphObject, Property, Label, RelatedObjects, RelatedTo, RelatedFrom
from py2neo import Graph, NodeSelector
graph = Graph()
selector = NodeSelector(graph)

In [2]:
url_base = 'https://i.weread.qq.com/'
#user
url_user_profile = '''https://i.weread.qq.com/user/profile?vid={}&gender=1&signature=1\
                        &vDesc=1&location=1&totalReadingTime=1&currentReadingTime=1&finishedBookCount=1\
                        &followerCount=1&followingCount=1&buyCount=1&reviewCount=1&reviewLikedCount=1\
                        &reviewCommentedCount=1&likeBookCount=1&isFollowing=1&isFollower=1&isBlackMe=1\
                        &isInBlackList=1&bookReviewCount=1&noteBookCount=1&exchangedMoney=0&recentReadingBooks=0\
                        &booklistCount=1&booklistCollectCount=1&articleBookId=1&articleCount=1&articleDraftCount=1\
                        &articleReadCount=1&articleSubscribeCount=1&articleWordCount=1&lectureCount=1&\
                        lectureListenCount=1&lectureLikedCount=1&lectureCommentedCount=1 
                    '''
url_book_finishreading = url_base+'book/finishreading?userVid={}'
url_shelf_friendsshelf = url_base+'shelf/friendsshelf?userVid={}'
#book
url_book_info = url_base+'book/info?bookId={}'
url_book_chapterInfos = url_base+'book/chapterInfos'
url_book_review_list = url_base+'review/list?listType=3&bookId={}&maxIdx=1000000&tips=1&listMode=1'
url_store_category_0 = url_base+'store/category?categoryId={}&synckey=0&count=1&maxIdx=0'
url_store_category_1 = url_base+'store/category?categoryId={}&synckey=0&maxIdx=0'
def get_from_api(api,id_):
    if api == url_book_chapterInfos:
        data = {"bookIds":[id_],"synckeys":[0]}
        r = requests.post(api,headers=headers,data=data)
    else:
        r = requests.get(api.format(id_),headers=headers)
    j = r.json()
    try:
        if j['errcode'] == -2014:
            print u'请求频率过高 ',
            time.sleep(15)
            return get_from_api(api,id_)
        if j['errcode'] == -2012:
            print u'登录超时 ',
            raise Exception(u'登录超时')
    except:
        return j

In [3]:
# url_review_sametimereading 
url_review_sametimereading_0 = url_base+'review/sametimereading?bookId={}&maxIdx=0&count=3'
url_review_sametimereading_1 = url_base+'review/sametimereading?bookId={}&count=100'
r = requests.get(url_review_sametimereading_0.format(852290),headers=headers)

In [4]:
j = r.json()
len(j['readingList'])
print j['readingList'][0]['review']['userVid'],j['readingList'][-1]['review']['userVid']

7511027 16125083


In [5]:
j

{u'maxShowCount': 50,
 u'readingList': [{u'finishedBookCount': 0,
   u'gender': 1,
   u'location': u'\u6c5f\u82cf \u5e38\u5dde',
   u'review': {u'author': {u'avatar': u'http://wx.qlogo.cn/mmopen/FicCThRvK2Iy9mTp1VUHCxd5z9Zg3TKUNib4Fu1KE92htNaj3XAbXJVpxhVqReFJXUKENnfn9ZOx8SDPgvgFcm8JLC5fXZ45zu/0',
     u'isFollower': 0,
     u'isFollowing': 0,
     u'name': u'\u5fc3\u7406\u4ea4\u6d41',
     u'userVid': 7511027},
    u'book': {u'author': u'\u5c24\u74e6\u5c14\xb7\u8d6b\u62c9\u5229',
     u'bookId': u'852290',
     u'bookStatus': 1,
     u'cover': u'http://wfqqreader.3g.qq.com/cover/290/852290/s_852290.jpg',
     u'format': u'epub',
     u'soldout': 0,
     u'title': u'\u672a\u6765\u7b80\u53f2\uff1a\u4ece\u667a\u4eba\u5230\u667a\u795e',
     u'version': 2017835511},
    u'bookId': u'852290',
    u'chapterIdx': 11,
    u'content': u'100',
    u'createTime': 1494608186,
    u'finishReading': 1,
    u'isPrivate': 0,
    u'noteCount': 514,
    u'readingTime': 44463,
    u'reviewId': u'7511027_

In [11]:
j['totalCount']/500

3

In [9]:
j = r.json()
len(j['readingList'])
print j['readingList'][0]['review']['userVid'],j['readingList'][-1]['review']['userVid']

30604467 30604467


In [19]:
def get_review_sametimereading(bookIds):
    for bookId in bookIds:
        print bookId,
        if booksrd.find_one({'_id':bookId}):
            pass
        else:
            #try:
            totalCount = get_from_api(url_review_sametimereading_0,bookId)['totalCount']
            print totalCount
            if totalCount > 0:
                books_data = {}
                books_data['_id'] = bookId
                books_data['totalCount'] = totalCount
                books_data['readingList'] = []
                for tc in xrange(totalCount/100+1):
                    print tc
                    books_data['readingList'].extend(get_from_api(url_review_sametimereading_1+'&maxIdx='+str(tc*100),bookId)['readingList'])
                #print url_store_category_1+'&count='+str(totalCount)
                #booksrd.insert_one(books_data)
            #except:
             #   pass

In [20]:
get_review_sametimereading([852290])

852290 1851
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
